In [72]:
# load model 
# import joblib
# ada_model = joblib.load('ada_model.pkl')
# scaler = joblib.load('scaler.pkl')

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
pd.set_option('display.max_columns', None)

In [75]:
df = pd.read_csv('kuisioner.csv')
dfc = df.copy()
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 29 columns):
 #   Column                                                                                                                                                                                   Non-Null Count  Dtype 
---  ------                                                                                                                                                                                   --------------  ----- 
 0   Timestamp                                                                                                                                                                                229 non-null    object
 1   Nama Lengkap                                                                                                                                                                             229 non-null    object
 2   Jenis Kelamin                         

In [76]:
iat_col = ["IAT-IAT_03","IAT-IAT_04","IAT-IAT_15","IAT-IAT_01","IAT-IAT_13","IAT-IAT_11","IAT-IAT_07",
           "IAT-IAT_02","IAT-IAT_18","IAT-IAT_06","IAT-IAT_19","IAT-IAT_17","IAT-IAT_14","IAT-IAT_16","IAT-IAT_12",
           "IAT-IAT_20","IAT-IAT_05","IAT-IAT_09","IAT-IAT_08","IAT-IAT_10"]
mapping_name = {
    'Jenis Kelamin': 'Basic_Demos-Sex',
    'Usia': 'Basic_Demos-Age'
}
dfc.rename(columns=mapping_name, inplace=True)
dfc.rename(columns=dict(zip(dfc.iloc[:,8:28].columns, iat_col)), inplace=True)
dfc.rename(columns={dfc.columns[-1]:'PreInt_EduHx-computerinternet_hoursday'}, inplace=True)

In [77]:
dfc['Berapa Tinggi Badan Anda?'] = dfc['Berapa Tinggi Badan Anda?'].str.extract(r'(\d+)').astype(float)
dfc['Berapa Berat Badan Anda?'] = dfc['Berapa Berat Badan Anda?'].str.extract(r'(\d+)').astype(float)

In [78]:
# Fill missing height values by gender
height_median_male = dfc.loc[dfc['Basic_Demos-Sex'] == 'Laki-laki', 'Berapa Tinggi Badan Anda?'].median()
height_median_female = dfc.loc[dfc['Basic_Demos-Sex'] == 'Perempuan', 'Berapa Tinggi Badan Anda?'].median()

dfc.loc[dfc['Basic_Demos-Sex'] == 'Laki-laki', 'Berapa Tinggi Badan Anda?'] = dfc.loc[dfc['Basic_Demos-Sex'] == 'Laki-laki', 'Berapa Tinggi Badan Anda?'].fillna(height_median_male)
dfc.loc[dfc['Basic_Demos-Sex'] == 'Perempuan', 'Berapa Tinggi Badan Anda?'] = dfc.loc[dfc['Basic_Demos-Sex'] == 'Perempuan', 'Berapa Tinggi Badan Anda?'].fillna(height_median_female)

# Fill missing weight values by gender
weight_median_male = dfc.loc[dfc['Basic_Demos-Sex'] == 'Laki-laki', 'Berapa Berat Badan Anda?'].median()
weight_median_female = dfc.loc[dfc['Basic_Demos-Sex'] == 'Perempuan', 'Berapa Berat Badan Anda?'].median()

dfc.loc[dfc['Basic_Demos-Sex'] == 'Perempuan', 'Berapa Berat Badan Anda?'] = dfc.loc[dfc['Basic_Demos-Sex'] == 'Perempuan', 'Berapa Berat Badan Anda?'].fillna(weight_median_female)
dfc.loc[dfc['Basic_Demos-Sex'] == 'Laki-laki', 'Berapa Berat Badan Anda?'] = dfc.loc[dfc['Basic_Demos-Sex'] == 'Laki-laki', 'Berapa Berat Badan Anda?'].fillna(weight_median_male)

In [79]:
dfc['Physical-BMI'] = dfc['Berapa Berat Badan Anda?'] / ((dfc['Berapa Tinggi Badan Anda?']/100) ** 2)
dfc['IAT-IAT_Total'] = dfc[iat_col].sum(axis=1)
dfc['Basic_Demos-Sex'] = dfc['Basic_Demos-Sex'].map({'Laki-laki': 0, 'Perempuan': 1})

In [80]:
col_to_drop = ['Timestamp', 'Nama Lengkap','Angkatan', 'NIM']
dfc.drop(columns=col_to_drop, inplace=True)
dfc.drop(['Berapa Tinggi Badan Anda?','Berapa Berat Badan Anda?'], axis=1, inplace=True)

In [81]:
index_col = ['Basic_Demos-Age', 'Basic_Demos-Sex', 'Physical-BMI', 'IAT-IAT_01','IAT-IAT_02','IAT-IAT_03','IAT-IAT_04','IAT-IAT_05',
             'IAT-IAT_06','IAT-IAT_07','IAT-IAT_08','IAT-IAT_09','IAT-IAT_10','IAT-IAT_11','IAT-IAT_12',
             'IAT-IAT_13','IAT-IAT_14','IAT-IAT_15','IAT-IAT_16','IAT-IAT_17','IAT-IAT_18','IAT-IAT_19',
             'IAT-IAT_20', 'IAT-IAT_Total', 'PreInt_EduHx-computerinternet_hoursday']
dfc = dfc.reindex(columns=index_col)

In [83]:
dfc.head()

,Basic_Demos-Age,Basic_Demos-Sex,Physical-BMI,IAT-IAT_01,IAT-IAT_02,IAT-IAT_03,IAT-IAT_04,IAT-IAT_05,IAT-IAT_06,IAT-IAT_07,IAT-IAT_08,IAT-IAT_09,IAT-IAT_10,IAT-IAT_11,IAT-IAT_12,IAT-IAT_13,IAT-IAT_14,IAT-IAT_15,IAT-IAT_16,IAT-IAT_17,IAT-IAT_18,IAT-IAT_19,IAT-IAT_20,IAT-IAT_Total,PreInt_EduHx-computerinternet_hoursday
0,22,1,20.549887,4,3,3,2,4,5,3,3,2,3,4,4,5,3,4,1,4,4,3,3,67,3
1,20,1,19.382145,4,1,3,2,5,0,4,1,3,0,4,2,4,4,4,0,3,1,5,1,51,3
2,21,0,22.679952,4,4,5,3,3,4,5,3,3,2,5,5,4,2,3,3,3,3,4,3,71,3
3,21,0,25.605536,4,1,3,3,3,1,4,4,3,4,4,3,4,3,4,3,3,1,2,2,59,3
4,22,1,27.040816,3,4,3,2,1,2,2,4,1,1,3,3,4,3,2,1,2,2,2,1,46,3
